In [180]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Dense # the two types of neural network layer we will be using
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""


In [181]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255 #Нормирование изображения
x_test = x_test / 255


In [182]:
x_train = tf.reshape(tf.cast(x_train, tf.float32),[-1,28*28])
x_text = tf.reshape(tf.cast(x_test, tf.float32),[-1,28*28])

In [183]:
y_train = to_categorical(y_train,10)


In [184]:
print('X_train: ' + str(x_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test:  '  + str(x_test.shape))
print('Y_test:  '  + str(y_test.shape))

X_train: (60000, 784)
Y_train: (60000, 10)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [185]:
class DenseNN(tf.Module): #Это слой
    def __init__(self, outputs, activate="relu"):
        super().__init__()
        self.outputs = outputs
        self.activate = activate
        self.fl_init = False

    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name="w")
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name="b")
            
        y = x @ self.w + self.b

        if self.activate == "relu":
            return tf.nn.relu(y) #
        elif self.activate == "softmax":
            return tf.nn.softmax(y) #
        
        return y

In [186]:
layer_1 = DenseNN(128)
layer_2 = DenseNN(10, activate="softmax")

def model_predict(x):
    y = layer_1(x)
    y = layer_2(y)
    return y

In [187]:
cross_entropy = lambda y_true, y_pred: tf.reduce_mean(tf.losses.categorical_crossentropy(y_true, y_pred))
opt = tf.optimizers.Adam(learning_rate=0.001)

In [188]:
BATCH_SIZE = 32 
EPOCHS = 10  #КОл-во эпох
TOTAL = x_train.shape[0] #Размер обучающей выборки

train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

In [189]:
#ЦИкл обучения
for n in range(EPOCHS): 
    loss = 0
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            f_loss = cross_entropy(y_batch, model_predict(x_batch))

        loss += f_loss
        grads = tape.gradient(f_loss,[layer_1.trainable_variables,layer_2.trainable_variables])
        opt.apply_gradients(zip(grads[0], layer_1.trainable_variables))
        opt.apply_gradients(zip(grads[1], layer_1.trainable_variables))

    print(loss.numpy())


4595.923
4594.116
4607.0


In [ ]:
y = model_predict(x_test)
y2 = tf.argmax(y, axis=1).numpy()
acc = len(y_test[y_test== y2])/y_test.shape[0] * 100
print(acc)

acc = tf.metrics.Accuracy()
acc.update_state(y_test, y2)
print(acc.result().numpy()*100)

C:\Users\mihee\AppData\Local\Temp\ipykernel_9432\706371978.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  acc = len(y_test[y_test== y2])/y_test.shape[0] * 100


0.0


ValueError: Shapes (10000,) and (10000, 10) are incompatible